In [ ]:
%%sh
pip -q install together

In [ ]:
import base64
import os
from together import Together
from IPython.display import Image

# Together.ai model identifier for Arcee AI Spotlight
model = "arcee_ai/arcee-spotlight"

# Get Together.ai API key from environment variable
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
if not TOGETHER_API_KEY:
    raise ValueError("Please set the TOGETHER_API_KEY environment variable")

client = Together(api_key=TOGETHER_API_KEY)

In [ ]:
from IPython.display import Markdown, clear_output, display

def print_streaming_response(response):
    num_tokens = 0
    content = ""
    for chunk in response:
        if hasattr(chunk, 'choices') and len(chunk.choices) > 0:
            if hasattr(chunk.choices[0], 'delta') and hasattr(chunk.choices[0].delta, 'content'):
                delta_content = chunk.choices[0].delta.content
                if delta_content:
                    num_tokens += 1
                    content += delta_content
                    clear_output(wait=True)
                    display(Markdown(content))
            elif hasattr(chunk.choices[0], 'message') and hasattr(chunk.choices[0].message, 'content'):
                msg_content = chunk.choices[0].message.content
                if msg_content:
                    num_tokens += 1
                    content += msg_content
                    clear_output(wait=True)
                    display(Markdown(content))

    print(f"\n\nNumber of tokens: {num_tokens}")


In [ ]:
# Define the URL and text for our examples
url = "https://img.lemde.fr/2019/07/14/0/0/4674/2832/556/0/75/0/b095b7e_be31f6a29f404568a2e861c48fc4a078-be31f6a29f404568a2e861c48fc4a078-0.jpg"
text = "Where was this picture taken?"

# display the image
display(Image(url))

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {"type": "image_url", "image_url": {"url": url}},
            ],
        }
    ],
    temperature=0.9,
    stream=True,
    max_tokens=2048,
)

print_streaming_response(response)

In [ ]:
text = "Write a short and precise caption for this picture"

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {"type": "image_url", "image_url": {"url": url}},
            ],
        }
    ],
    temperature=0.9,
    stream=True,
    max_tokens=2048,
)

print_streaming_response(response)


In [ ]:
text = """Generate JSON metadata for this picture:
- Country
- City
- Landmarks
- Short description
- Detailed description
- Themes
- Keywords
"""

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {"type": "image_url", "image_url": {"url": url}},
            ],
        }
    ],
    temperature=0.9,
    stream=True,
    max_tokens=2048,
)

print_streaming_response(response)


In [ ]:
def load_image_as_base64(file_path):
    # Load the image as a base64 string
    with open(file_path, "rb") as image_file:
        base64_string = base64.b64encode(image_file.read()).decode("utf-8")
    return base64_string


In [ ]:
image_file = "july14th.jpg"
image_base64 = load_image_as_base64(image_file)

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"},
                },
            ],
        }
    ],
    temperature=0.9,
    stream=True,
    max_tokens=2048,
)

print_streaming_response(response)


In [ ]:
image1 = "july14th.jpg"
image2 = "italy.png"

# Load both images as base64
july14th_base64 = load_image_as_base64(image1)
july14th_2_base64 = load_image_as_base64(image2)

# Display both images
display(Image(image1))
print("\n")
display(Image(image2))

In [ ]:
text = """Compare these two images of the French Bastille Day parade:
- What similarities do you see?
- What differences do you notice?
- Are they from the same country or not
- What specific details support your conclusion?"""

# Create a message with both images
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{july14th_base64}"},
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{july14th_2_base64}"},
                }
            ],
        }
    ],
    temperature=0.9,
    stream=True,
    max_tokens=2048,
)

print_streaming_response(response)
